In [242]:
# Import packages
import gurobipy as gp
from gurobipy import GRB
import parse_graph_new
import networkx as nx
import path_enumeration
import math

In [243]:
# Variablen (sollen der Funktion übergeben werden)
norm = "L1"
sparsity_constr = "L0"
factor = 10000

In [244]:
# Read file and extract all paths
with open("/home/laura/Documents/Transkript_Assembly/data/human_geuvadis/test2.graph") as f: 
    fileEndReached = False
    f.readline()
    while not fileEndReached:
        f.readline()
        Chromosome, Strand, Exons = parse_graph_new.parse_meta(f)
        Bins = parse_graph_new.parse_bins(f)
        PairedBins = parse_graph_new.parse_pairs(f)
        
        # Build graphs
        G_full = nx.DiGraph()
        fileEndReached, skip = parse_graph_new.parse_graph(f, G_full, Exons) # Full Graph
        
        if not fileEndReached and not skip:
            G_clean = nx.DiGraph()  
            fileEndReached, _ = parse_graph_new.parse_graph(f, G_clean, Exons) # Cleaned Graph
            
        # Full path enumeration of cleaned graph
        transcripts = path_enumeration.enumeration(G_clean,[],"0",["0"],"1",False)
        print("Transkripts:", len(transcripts), transcripts)

f.close()

Transkripts: 2 [['0', '2', '19', '3', '18', '4', '17', '5', '16', '1'], ['0', '2', '19', '3', '18', '4', '17', '7', '14', '9', '12', '10', '11', '1']]


In [245]:
# Create edge dictionary storing the counts for each edge
edges_dict = {}
for edgeKey, edgeValue in G_clean.edges.items():
    count = edgeValue["counts"]["c"]
    if edgeValue["type"] == "SpliceJunction" or edgeValue["type"] == "Exon":
        edges_dict[edgeKey] = count
print(edges_dict)
edges = list(edges_dict.keys())
print(len(edges))


{('19', '3'): 174, ('3', '18'): 216, ('18', '4'): 163, ('4', '17'): 227, ('17', '5'): 49, ('17', '7'): 129, ('5', '16'): 89, ('7', '14'): 146, ('14', '9'): 28, ('9', '12'): 53, ('12', '10'): 53, ('10', '11'): 53, ('2', '19'): 211}
13


In [246]:
# Transkripte
print(transcripts)

[['0', '2', '19', '3', '18', '4', '17', '5', '16', '1'], ['0', '2', '19', '3', '18', '4', '17', '7', '14', '9', '12', '10', '11', '1']]


In [247]:
# Create new Gurobi model
model = gp.Model("Transcript Expression")

In [248]:
# Add variables
no_trans = len(transcripts)
var1 = model.addVars(no_trans, vtype=GRB.CONTINUOUS, name="expression") # Expression levels
helper1 = model.addVars(edges, lb=-GRB.INFINITY, vtype=GRB.CONTINUOUS, name="x") # Innerer Term

if norm == "L1":
    norm1 = model.addVars(edges, vtype=GRB.CONTINUOUS, name="L1_norm") # L1 Norm
elif norm == "L0":
    norm0 = model.addVar(name="L0_norm") # L0 norm
elif norm == "L2":
    norm2_1 = model.addVars(edges, vtype=GRB.CONTINUOUS, name="L2_norm_1") # Absolute value of helper1
    norm2_2 = model.addVars(edges, vtype=GRB.CONTINUOUS, name="L2_norm_2") # Quadrat von norm2_1
    norm2_3 = model.addVar(name="L2_norm_3") # Summe von norm2_2
    norm2_4 = model.addVar(name="L2_norm_3") # Wurzel von norm2_3
    
if sparsity_constr == "L0":
    sparsity_norm0 = model.addVar(name="Sparsity_Constraint_L0") # Sparsity Constraint L0 Norm

print(vars)

<built-in function vars>


In [249]:
# Create Adjazenzmatrix (path,edge): 0/1
adj_matrix = {}
for i in range(0,len(transcripts)):
    for j in range(1,len(transcripts[i])-1):
        startnode = transcripts[i][j] 
        endnode = transcripts[i][j+1]
        current_edge = (startnode,endnode)
        if current_edge in edges:
            adj_matrix[i,current_edge] = 1
    for edge in edges:
        if (i,edge) not in adj_matrix.keys():
            adj_matrix[i,edge] = 0
print(adj_matrix)

{(0, ('2', '19')): 1, (0, ('19', '3')): 1, (0, ('3', '18')): 1, (0, ('18', '4')): 1, (0, ('4', '17')): 1, (0, ('17', '5')): 1, (0, ('5', '16')): 1, (0, ('17', '7')): 0, (0, ('7', '14')): 0, (0, ('14', '9')): 0, (0, ('9', '12')): 0, (0, ('12', '10')): 0, (0, ('10', '11')): 0, (1, ('2', '19')): 1, (1, ('19', '3')): 1, (1, ('3', '18')): 1, (1, ('18', '4')): 1, (1, ('4', '17')): 1, (1, ('17', '7')): 1, (1, ('7', '14')): 1, (1, ('14', '9')): 1, (1, ('9', '12')): 1, (1, ('12', '10')): 1, (1, ('10', '11')): 1, (1, ('17', '5')): 0, (1, ('5', '16')): 0}


In [250]:
# Optimization problem

# Sparsity Constraints
if sparsity_constr == "L1":
    model.addConstr(var1.sum() <= factor)
elif sparsity_constr == "L0":
    model.addGenConstrNorm(sparsity_norm0, var1, 0) # Hinzufügen dieser Zeile verändert sofort die Werte der Variablen var1, warum?
    #model.setObjective((gp.quicksum(helper2[j] for j in edges) + (factor*sparsity_norm0)), GRB.MINIMIZE)
    model.addConstr(sparsity_norm0 <= factor)

# Normen
if norm == "L1":
    for j in edges:
        model.addConstr(helper1[j] == (edges_dict[j] - (gp.quicksum(adj_matrix[i,j] * var1[i] for i in range(len(transcripts))))))
        model.addConstr(norm1[j] >= helper1[j])
        model.addConstr(norm1[j] >= -helper1[j])
    model.setObjective(gp.quicksum(norm1[j] for j in edges), GRB.MINIMIZE)

elif norm == "L0":
    for j in edges:
        model.addConstr(helper1[j] == (edges_dict[j] - (gp.quicksum(adj_matrix[i,j] * var1[i] for i in range(len(transcripts))))))
    model.addGenConstrNorm(norm0, helper1, 0)
    model.setObjective(norm0, GRB.MINIMIZE)

elif norm == "L2":
    for j in edges:
        model.addConstr(helper1[j] == (edges_dict[j] - (gp.quicksum(adj_matrix[i,j] * var1[i] for i in range(len(transcripts))))))
        model.addConstr(norm2_1[j] >= helper1[j])
        model.addConstr(norm2_1[j] >= -helper1[j])
        model.addGenConstrPow(norm2_1[j], norm2_2[j], 2)
    model.addConstr(norm2_3 == gp.quicksum(norm2_2[j] for j in edges))
    model.addGenConstrPow(norm2_3, norm2_4, 0.5)
    model.setObjective(norm2_4, GRB.MINIMIZE)
    


In [251]:
# Solve optimization problem
model.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 39 rows, 29 columns and 83 nonzeros
Model fingerprint: 0xcf602694
Model has 1 general constraint
Variable types: 29 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 2e+02]
Presolve removed 16 rows and 15 columns
Presolve time: 0.00s
Presolved: 23 rows, 14 columns, 47 nonzeros
Variable types: 14 continuous, 0 integer (0 binary)

Root relaxation: objective 4.410000e+02, 11 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     441.0000000  441.00000  0.00%     -    0s

Explored 1 nodes (11 simplex iterations) in 0.02 seconds (0.00 

In [252]:
# Print results
for var in model.getVars():
    print(var.VarName)
    print(var.X)

expression[0]
121.0
expression[1]
53.0
x[19,3]
-0.0
x[3,18]
42.0
x[18,4]
-11.0
x[4,17]
53.0
x[17,5]
-72.0
x[17,7]
76.0
x[5,16]
-32.0
x[7,14]
93.0
x[14,9]
-25.0
x[9,12]
0.0
x[12,10]
0.0
x[10,11]
0.0
x[2,19]
37.0
L1_norm[19,3]
0.0
L1_norm[3,18]
42.0
L1_norm[18,4]
11.0
L1_norm[4,17]
53.0
L1_norm[17,5]
72.0
L1_norm[17,7]
76.0
L1_norm[5,16]
32.0
L1_norm[7,14]
93.0
L1_norm[14,9]
25.0
L1_norm[9,12]
0.0
L1_norm[12,10]
0.0
L1_norm[10,11]
0.0
L1_norm[2,19]
37.0
Sparsity_Constraint_L0
2.0


In [253]:
for i in var1:
    print(var1[i].X)

121.0
53.0


In [254]:
model.reset()

Discarded solution information
